## 📦 Analyzing MySQL Data with Pandas and Plotly

##### 🔷 Importing Required Libraries

In [1]:
import pandas as pd                     # For data manipulation and analysis
import numpy as np                      # For numerical operations
import  plotly.express as px            # For interactive visualizations
from sqlalchemy import create_engine    # For connecting to MYSQL databases

In [2]:
from dotenv import load_dotenv
import os
load_dotenv() 
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
database = os.getenv("DB_NAME")

In [3]:
import plotly.io as pio                  # Control Plotly rendering/output
# Set renderer to PNG (static image)
#pio.renderers.default = 'iframe'

##### 🔷 Set Up SQLAlchemy Engine to Connect to MySQL

In [4]:
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

##### 🔷 SQL query to fetch all rows and columns from the 'ppm' table

In [5]:
query ="select * from ppm;"     

##### 🔷 Execute the query and load the result into a DataFrame

In [6]:
df = pd.read_sql(query, engine)   

##### 🔷 First 5 Rows of the DataFrame

In [7]:
df.head()                           

,Date,Shift,Part_No,Production,Screen_Overlap_F,Dent_F,Poor_Paintwork_F,Screen_Fail_F,Dv_Damage_F,Dv_Wire_Cut_F,...,Dv_Fitment_Fail_F,Dv_Loose_F,Without_PPI_Inspection_P,Dv_Torque_Marking_Miss_F,Washer_Sealing_Leak_P,Connector_Damage_P,Washer_Sealing_R,Adaptor_Damage_P,Hpp_Pump_Fail_R,F_Scratch_Mark_P
0,2024-07-02,I,8539665,60,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2024-07-02,I,8059201,300,0,1,131,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2024-07-02,I,8185559,90,0,6,10,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2024-07-02,I,8567029,90,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2024-07-02,I,5046557,90,0,0,56,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 🔷 Check DataFrame Schema and Non-Null Counts

In [8]:
df.info()           

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3180 entries, 0 to 3179
Data columns (total 45 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Date                      3180 non-null   datetime64[ns]
 1   Shift                     3180 non-null   object        
 2   Part_No                   3180 non-null   object        
 3   Production                3180 non-null   int64         
 4   Screen_Overlap_F          3180 non-null   int64         
 5   Dent_F                    3180 non-null   int64         
 6   Poor_Paintwork_F          3180 non-null   int64         
 7   Screen_Fail_F             3180 non-null   int64         
 8   Dv_Damage_F               3180 non-null   int64         
 9   Dv_Wire_Cut_F             3180 non-null   int64         
 10  Element_Miss_R            3180 non-null   int64         
 11  Seam_Leak_F               3180 non-null   int64         
 12  Crimp_Nut_Leak_F    

##### 🔷 Select Relevant Columns for Analysis

In [9]:
prod= df[['Date','Shift','Part_No','Production']]           

##### 🔷 Check DataFrame Schema and Non-Null Counts

In [10]:
prod.info()          

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3180 entries, 0 to 3179
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        3180 non-null   datetime64[ns]
 1   Shift       3180 non-null   object        
 2   Part_No     3180 non-null   object        
 3   Production  3180 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 99.5+ KB


##### 🔷 Convert `Date` Column to Datetime Format and Verify Type

In [11]:
prod.loc[:,'Date'] = pd.to_datetime(prod['Date'])
print("Type of Date:", prod['Date'].dtype)

Type of Date: datetime64[ns]


##### 🔷 Calculate and Print Total Production Quantity

In [12]:
prod['Production'].sum()
print("Total Production Qty:",prod['Production'].sum())

Total Production Qty: 272858


##### 🔷 Calculate Mean Daily Production

In [13]:
Daily_mean= int(prod.groupby('Date')['Production'].sum().mean())
print(f'Mean:{Daily_mean}')

Mean:1819


##### 🔷 Calculate Min, Max, and Mean

In [14]:
prod.groupby('Date')['Production'].sum().agg(['min','max','mean']).astype(int)

min      510
max     2784
mean    1819
Name: Production, dtype: int64

##### 🔷 Total Daily Production

In [15]:
daily_prod = df.groupby('Date')['Production'].sum().reset_index()

##### 🔷 Plot Daily Total Production Using a Line Chart

In [16]:
fig1= px.line(daily_prod, 
               x='Date',
            y= 'Production',
            title='Daily total Production',
            
            )
fig1.show()

##### 🔷 Total Production by Shift

In [17]:
Shift_prod = df.groupby('Shift')['Production'].sum().reset_index()
Shift_prod

,Shift,Production
0,I,182498
1,II,90360


##### 🔷 Production by Shift Using a Bar Chart

In [18]:
fig2 =px.bar(Shift_prod,x='Shift',y='Production',title='Production by Shift',color='Shift', text_auto=True)
fig2.show()

##### 🔷 Top 5 Parts with Highest Production

In [19]:
top5_Parts = prod.groupby('Part_No')['Production']\
            .sum()\
            .reset_index()\
            .sort_values(by='Production',ascending=False)\
            .head(5)\
            .reset_index(drop=True)
print("Top 5 Productions\n",top5_Parts)

Top 5 Productions
    Part_No  Production
0  8059201       39500
1  7800359       22197
2  8397572       19333
3  7488374       13376
4  5905916       12144


##### 🔷 Top 5 Produced Parts Using a Pie Chart

In [20]:
fig3= px.pie(top5_Parts,names='Part_No',values='Production',title="Top 5 Produced Parts")
fig3.show()

#### 🔷 Summary
##### This notebook shows a simple analysis of production data from a MySQL database called production. It uses Python tools like pandas, plotly, and sqlalchemy to read and work with the data. The main focus is on understanding how much production was done each day, in different shifts, and which parts were made the most.First, the data is cleaned and dates are set in the right format. Then, the total and average daily production is calculated. A line graph shows how production changes day by day. A bar chart compares production between shifts, helping to see which shift performs better. Finally, a pie chart shows the top 5 parts that were produced the most.
##### Overall, this analysis helps to understand production trends and where improvements can be made.

In [21]:
#! jupyter nbconvert --to html "A:\Data Analyst\Python\CWH\Code\Project_pandas\sql_local\production_6month.ipynb"

